In [122]:
%run -i motor_construction_no_pv

In [123]:

from bluesky.plan_stubs import null, mv

import os, subprocess, inspect
from rich import print as cprint
# from motor_construction import SlitsWithLookup
from ophyd.sim import make_fake_device, SynAxis, FakeEpicsSignal
# from motor_construction_no_slt_pv import SlitsWithLookup

sim_SlitsWithLookup = make_fake_device(SlitsWithLookup)
sim_motor = make_fake_device(MotorWithLookup)

slt3WithLookup = sim_SlitsWithLookup('XF:23ID1-OP{Slt:3', name='slt3WithLookup')
es_diagWithLookup = sim_motor("XF:23ID1-ES{Diag:1-Ax:Y}", name = "es_diagWithLookup")


In [124]:
slt3WithLookup.get_all_sizes()
es_diagWithLookup.get_all_positions()

4 possible sizes:
----------------------------------
    2000______ : (8.8, 0.0)
    50________ : (0.8, 0.0)
    20________ : (-8.727, 0.05)
    10________ : (-17.35, -0.075)
10 possible positions:
----------------------------------
    Out____________ : -55.0
    YAG____________ : -62.0
    Cu Block_______ : -74.0
    SrTiO3_________ : -76.0
    HEO-dark_______ : -79.0
    HEO-light______ : -82.0
    MgAl2O4________ : -86.0
    Si_____________ : -89
    ZnSe___________ : -92.0
    Undefined______ : 0.0


In [125]:
def colored(text, tint='white', attrs=[], end=None):
    '''
    A simple wrapper around rich.print
    '''
    if not False:
        tint = tint.lower()
        this = f'[{tint}]{text}[/{tint}]'
        if end is not None:
            cprint(f'[{tint}]{text}[/{tint}]', end=end)
        else:
            cprint(f'[{tint}]{text}[/{tint}]')
    else:
        print(text)


In [126]:
def error_msg(text, end=None):
    '''Red text'''
    colored(text, 'red1', end=end)
def warning_msg(text, end=None):
    '''Yellow text'''
    colored(text, 'yellow', end=end)
def go_msg(text, end=None):
    '''Green text'''
    colored(text, 'green', end=end)
def url_msg(text, end=None):
    '''Underlined text, intended for URL decoration...'''
    colored(text, 'underline', end=end)
def bold_msg(text, end=None):
    '''Bright yellow text'''
    colored(text, 'yellow2', end=end)
def verbosebold_msg(text, end=None):
    '''Bright cyan text'''
    colored(text, 'cyan', end=end)
def list_msg(text, end=None):
    '''Dark cyan text'''
    colored(text, 'bold cyan', end=end)
def disconnected_msg(text, end=None):
    '''Purple text'''
    colored(text, 'magenta3', end=end)
def info_msg(text, end=None):
    '''Brown text'''
    colored(text, 'light_goldenrod2', end=end)
def cold_msg(text, end=None):
    '''Light blue text'''
    colored(text, 'blue', end=end)
def whisper(text, end=None):
    '''Light gray text'''
    colored(text, 'bold black', end=end)


In [135]:
class WDYWTD():
    '''What Do You Want To Do?

    An extremely simple textual user interface to the most commonly
    performed chores at BMM.

    '''
    def wdywtd(self):
        '''Prompt the user to do a thing.
        '''

        actions = {'1': ('MoveDiag',         'move es_diag1'),
                   '2': ('ChangeEdge',   'change edge'),
                   '3': ('Spreadsheet',  'import spreadsheet'),
                   '4': ('RunMacro',     'run macro'),
                   '5': ('AlignSlot',    'align wheel slot'),
                   '6': ('AlignGA',      'align glancing angle stage'),
                   '7': ('XRFSpectrum',  'view XRF spectrum'),

                   'A': ('RockingCurve', 'measure rocking curve'),
                   'B': ('SlitHeight',   'measure slit height'),
                   'C': ('AdjustSlit3',  'adjust slit size'),
                   'D': ('ChangeXtals',  'change monochrometer crystals'),
                   'E': ('SetupXRD',     'prepare for XRD'),
        }
        
        print('''
  COMMON CHORES                            OTHER CHORES
================================================================================''')

        for i in range(1,8):
            text  = actions[str(i)][1]
            try:
                other = actions[chr(64+i)][1]
                print(f' {i}. {text:37} {chr(64+i)}. {other:37}')
            except:
                print(f' {i}. {text:37}')
        print('')
        choice = input(" What do you want to do? ")
        choice = choice.upper()
        print('\n')
        def bailout():
            whisper('doing nothing')
            yield from null()
        thing = 'do_nothing'
        if choice in actions:
            thing = f'do_{actions[choice][0]}'
        return getattr(self, thing, lambda: bailout)()
    
    def do_MoveDiag(self):
        go_msg('You would like to move es_diag1...\n')
        print("Where would you like to move it? (position name or 'm' for manual value)")
        es_diagWithLookup.get_all_positions()
        where = input()
        print(f"You chose {where}")
        y = float(es_diagWithLookup.motor.read()['es_diagWithLookup_motor']['value'])
        match where:
            case "m":
                print("Enter Y value (number or 'c' for current value): ")
                i = input()
                match i:
                    case "c":
                        y = y
                    case _:
                        y = float(i)
            case _:
                try:
                    y = es_diagWithLookup.lookup(where)
                except ValueError:
                    print(f"{where} is not one of the available options")
                    return
        print(f"Y: {y}")
        print(f"CONFIRM: Move to {y} (y/n)")
        confirmation = input()
        match confirmation.lower():
            case "y" | "yes":
                print (f"Moving to {y}...")
                print ("yield from mv(es_diagWithLookup, y)")
            case "n" | "no":
                print ("Movement canceled.")
        
    
    def do_AdjustSlit3(self):
        go_msg('You would like to adjust the size of the slt3...\n')
        print("Where would you like to position it? (2000, 50, 20, 10, or 'm' for manual coordinates)")
        slt3WithLookup.get_all_sizes()
        where = input()
        where = where.lower()
        print(f"You chose {where}")
        x = float(slt3WithLookup.x.read()['slt3WithLookup_x']['value'])
        y = float(slt3WithLookup.y.read()['slt3WithLookup_y']['value'])
        match where:
            case "2000" | "50" | "20" | "10":
                x, y = slt3WithLookup.lookup(where)
            case "m":
                print("Enter X value (number or 'c' for current value): ")
                i = input()
                match i:
                    case "c":
                        x = x
                    case _:
                        x = float(i)
                print(f"X: {x}")
                print("Enter Y value (number or 'c' for current value): ")
                i = input()
                match i:
                    case "c":
                        y = y
                    case _:
                        y = float(i)
                print(f"Y: {y}")
            case _:
                print(f"{where} is not one of the available options")
                return
        
        print(f"CONFIRM: Move to ({x}, {y}) (y/n)")
        confirmation = input()
        match confirmation.lower():
            case "y" | "yes":
                print (f"Moving to ({x}, {y})...")
                print ("yield from mv(slt3WithLookup, (x, y))")
            case "n" | "no":
                print ("Movement canceled.")
            

            

In [138]:
prompt = WDYWTD()

prompt.wdywtd()


  COMMON CHORES                            OTHER CHORES
 1. move es_diag1                         A. measure rocking curve                
 2. change edge                           B. measure slit height                  
 3. import spreadsheet                    C. adjust slit size                     
 4. run macro                             D. change monochrometer crystals        
 5. align wheel slot                      E. prepare for XRD                      
 6. align glancing angle stage           
 7. view XRF spectrum                    





You would like to move es_diag1...

Where would you like to move it? (position name or 'm' for manual value)
10 possible positions:
----------------------------------
    Out____________ : -55.0
    YAG____________ : -62.0
    Cu Block_______ : -74.0
    SrTiO3_________ : -76.0
    HEO-dark_______ : -79.0
    HEO-light______ : -82.0
    MgAl2O4________ : -86.0
    Si_____________ : -89
    ZnSe___________ : -92.0
    Undefined______ : 0.0
You chose m
Enter Y value (number or 'c' for current value): 
Y: 0.0
CONFIRM: Move to 0.0 (y/n)
Moving to 0.0...
yield from mv(es_diagWithLookup, y)
